<a href="https://colab.research.google.com/github/TuliDas/Classification-of-Sentimental-Reviews/blob/main/4_1_Sentiment_Analysis_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Importing Libraries***

In [ ]:
import numpy as np
import pandas as pd

***Importing Dataset***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/5. Sentiment Analysis
!ls

/content/drive/MyDrive/Colab Notebooks/5. Sentiment Analysis
'4.1 Sentiment Analysis Model .ipynb'   a1_RestaurantReviews_HistoricDump.tsv
'4.2 Sentiment_Predictor.ipynb'         a2_RestaurantReviews_FreshDump.tsv


In [ ]:
dataset = pd.read_csv('a1_RestaurantReviews_HistoricDump.tsv', delimiter = '\t', quoting = 3)

In [ ]:
dataset.shape

(900, 2)

In [ ]:
dataset.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


Data PreProcessing or cleaning.

Stop Words: do not carry much meaningful information about the content of the text. Example :  "the", "is", "and", "in", "of", "to", "a", "for", "with", "on", "Which", "Who" etc

Step Words: Stemming is the process of reducing a word to its base or root form. Example: Playing -> Play ; Recommended -> Recommend

In [ ]:
import re            # regular expression
import nltk          # Natural Language Toolkit

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

***Remove all stop words, punctuations, digits and stem the words***

In [ ]:
corpus=[]

for i in range(0, 900):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus

Data Transformation

Build Bag of words

The Bag of Words (BoW) model is a commonly used technique in natural language processing (NLP) for representing text data as numerical feature vectors.

3 steps.
1. Tokenaization : The first step is to break down the text into individual words or tokens.
2. Vocabulary Creation : a vocabulary is constructed by collecting all unique words from the entire corpus of documents. Each unique word becomes a feature in the vocabulary.
3. Vectorization : For each document in the corpus, a feature vector is created. The length of the feature vector is equal to the size of the vocabulary, and each element in the vector represents the frequency or presence of a word from the vocabulary in the document.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1420)

# 1420 are max number of token

In [ ]:
# X = bag of words , Y = histotical labels

X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

In [ ]:
# Saving Bag of Words dictionary in Drive to later use in prediction
import pickle
bow_path = 'c1_BoW_Sentiment_Model.pkl'
pickle.dump(cv, open(bow_path, "wb"))

Dividing Dataset into Training and testing set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

**Naive Bayes Classiifer for Model fitting**

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [ ]:
# Exporting Naive Bayes Classifier to Drive to later use in prediction
import joblib
joblib.dump(classifier, 'c2_Classifier_Sentiment_Model')

['c2_Classifier_Sentiment_Model']

In [ ]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)

[[67 11]
 [38 64]]


0.7277777777777777